In [6]:
import json
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, Activation, Embedding, Flatten, SpatialDropout1D
from tensorflow.keras.preprocessing.text import one_hot
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import json
from keras.models import model_from_json

# Import libraries
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
# nltk.download()
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)
# import these modules 
from nltk.stem import WordNetLemmatizer 

In [7]:
#!pip install geopandas

import pandas as pd
import numpy as np
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [8]:
locator = Nominatim(user_agent="myGeocoder")
location1 = locator.geocode("Braamfontein, Johannesburg")
location2 = locator.geocode("Melville, Johannesburg")

In [9]:
# print(location.address)
print("Latitude = {}, Longitude = {}".format(location1.latitude, location1.longitude))
print("Latitude = {}, Longitude = {}".format(location2.latitude, location2.longitude))

Latitude = -26.1923213, Longitude = 28.0361983
Latitude = -26.1751628, Longitude = 28.0108596


In [10]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

In [11]:
print(haversine_distance(location1.latitude, location1.longitude,location2.latitude, location2.longitude),'km')

3.17 km


In [12]:
data = pd.read_csv('nlp-getting-started/train.csv')
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [13]:
nltk.download('stopwords')
#### removing entries where location values are missing
data_after_null_removal = data.copy()
data_after_null_removal = data_after_null_removal.dropna(subset=['location'])

#punctutation removal
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

data_after_punctuation_removal = data_after_null_removal.copy()
data_after_punctuation_removal['text'] = data_after_punctuation_removal['text'].apply(lambda x: clean_text(x))
data_after_punctuation_removal['location'] = data_after_punctuation_removal['location'].apply(lambda x: clean_text(x))


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/patrick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:

# Tockenization

def tokenization(text):
    text = re.split('\W+', text)
    return text

tockenized_data = data_after_punctuation_removal.copy()
tockenized_data['text'] = tockenized_data['text'].apply(lambda x: tokenization(x.lower()))
tockenized_data['location'] = tockenized_data['location'].apply(lambda x: tokenization(x.lower()))

# stopword removal 
data_after_stopword_removal = tockenized_data.copy()
stopword = nltk.corpus.stopwords.words('english')

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

data_after_stopword_removal['text'] = data_after_stopword_removal['text'].apply(lambda x: remove_stopwords(x))
data_after_stopword_removal['location'] = data_after_stopword_removal['location'].apply(lambda x: remove_stopwords(x))

# return to string
data_without_tockenization = data_after_stopword_removal.copy()
def listToString(s):     
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 
        
data_without_tockenization['text'] = data_without_tockenization['text'].apply(lambda x: listToString(x))
data_without_tockenization['location'] = data_without_tockenization['location'].apply(lambda x: listToString(x))

data_without_tockenization
# Removing duplicates
#data_after_duplicate_removal = data_without_tockenization.drop_duplicates(subset=['text'])
#data_after_duplicate_removal.dropna(subset=['location'], inplace=True)
#data_after_duplicate_removal.head()

,id,keyword,location,text,target
31,48,ablaze,birmingham,bbcmtd wholesale markets ablaze,1
32,49,ablaze,est september bristol,always try bring heavy metal rt,0
33,50,ablaze,africa,africanbaze breaking newsnigeria flag set ablaze aba,1
34,52,ablaze,philadelphia pa,crying set ablaze,0
35,53,ablaze,london uk,plus side look sky last night ablaze,0
...,...,...,...,...,...
7575,10826,wrecked,tn,bright side wrecked,0
7577,10829,wrecked,newcastleupontyne uk,hes gone relax thought wife wrecked cake goner mind lol whoops,0
7579,10831,wrecked,vancouver canada,three days work theyve pretty much wrecked hahaha shoutout family one,0
7580,10832,wrecked,london,fx forex trading cramer igers words wrecked disneys stock,0


In [15]:
def clean(tweet):

    #correct some acronyms while we are at it
    tweet = re.sub("tnwx", "tennessee weather", tweet)
    tweet = re.sub(r"azwx", "arizona weather", tweet)  
    tweet = re.sub(r"alwx", "alabama weather", tweet)
    tweet = re.sub(r"wordpressdotcom", "wordpress", tweet)      
    tweet = re.sub(r"gawx", "georgia weather", tweet)  
    tweet = re.sub(r"scwx", "south carolina weather", tweet)  
    tweet = re.sub(r"cawx", "california weather", tweet)
    tweet = re.sub(r"usnwsgov", "united states national weather service", tweet) 
    tweet = re.sub(r"MH370", "malaysia airlines flight 370", tweet)
    tweet = re.sub(r"okwx", "oklahoma city weather", tweet)
    tweet = re.sub(r"arwx", "arkansas weather", tweet)  
    tweet = re.sub(r"lmao", "laughing my ass off", tweet)  
    tweet = re.sub(r"amirite", "am i right", tweet)
    
    #and some typos/abbreviations
    tweet = re.sub(r"w/e", "whatever", tweet)
    tweet = re.sub(r"w/", "with", tweet)
    tweet = re.sub(r"usagov", "usa government", tweet)
    tweet = re.sub(r"recentlu", "recently", tweet)
    tweet = re.sub(r"ph0tos", "photos", tweet)
    tweet = re.sub(r"exp0sed", "exposed", tweet)
    tweet = re.sub(r"<3", "love", tweet)
    tweet = re.sub(r"amageddon", "armageddon", tweet)
    tweet = re.sub(r"trfc", "traffic", tweet)
    tweet = re.sub(r"windStorm", "wind storm", tweet)
    tweet = re.sub(r"16yr", "16 year", tweet)
    tweet = re.sub(r"traumatised", "traumatized", tweet)
    
    #hashtags and usernames
    tweet = re.sub(r"irandeal", "iran deal", tweet)
    tweet = re.sub(r"arianagrande", "ariana grande", tweet)
    tweet = re.sub(r"camilacabello97", "camila cabello", tweet) 
    tweet = re.sub(r"rondarousey", "ronda rousey", tweet)     
    tweet = re.sub(r"mtvhottest", "mtv hottest", tweet)
    tweet = re.sub(r"trapmusic", "trap music", tweet)
    tweet = re.sub(r"prophetmuhammad", "prophet Mmuhammad", tweet)
    tweet = re.sub(r"PantherAttack", "panther attack", tweet)
    tweet = re.sub(r"strategicpatience", "strategic patience", tweet)
    tweet = re.sub(r"socialnews", "social news", tweet)
    tweet = re.sub(r"idps:", "internally displaced people :", tweet)
    tweet = re.sub(r"artistsUnited", "artists united", tweet)
    tweet = re.sub(r"claytonBryant", "clayton bryant", tweet)
    tweet = re.sub(r"uk", "united kingdom", tweet)
    tweet = re.sub(r"usa", "united states of america", tweet)
    tweet = re.sub(r"ny", "new york", tweet)
    tweet = re.sub(r"newcastleupontyne", "newcastle upon tyne", tweet)
    tweet = re.sub(r"jimmyfallon", "jimmy fallon", tweet)
    tweet = re.sub(r"justinbieber", "justin bieber", tweet)
    tweet = re.sub(r"time2015", "time 2015", tweet)
    tweet = re.sub(r"djicemoon", "dj icemoon", tweet)
    tweet = re.sub(r"livingsafely", "living safely", tweet)
    tweet = re.sub(r"fifa16", "fifa 2016", tweet)
    tweet = re.sub(r"thisiswhywecanthavenicethings", "this is why we cannot have nice things", tweet)
    tweet = re.sub(r"bbcnews", "bbc news", tweet)
    tweet = re.sub(r"undergroundrailraod", "underground railraod", tweet)
    tweet = re.sub(r"c4news", "c4 news", tweet)
    tweet = re.sub(r"mudslide", "mud slide", tweet)
    tweet = re.sub(r"nosurrender", "no surrender", tweet)
    tweet = re.sub(r"notexplained", "not explained", tweet)
    tweet = re.sub(r"greatbritishbakeoff", "great british bake off", tweet)
    tweet = re.sub(r"londonfire", "london fire", tweet)
    tweet = re.sub(r"kotaweather", "kota weather", tweet)
    tweet = re.sub(r"luchaunderground", "lucha underground", tweet)
    tweet = re.sub(r"koin6news", "koin 6 news", tweet)
    tweet = re.sub(r"liveOnK2", "live on K2", tweet)
    tweet = re.sub(r"9newsgoldcoast", "9 news gold coast", tweet)
    tweet = re.sub(r"nikeplus", "nike plus", tweet)
    tweet = re.sub(r"david_cameron", "david cameron", tweet)
    tweet = re.sub(r"peterjukes", "peter jukes", tweet)
    tweet = re.sub(r"mikeparractor", "michael parr", tweet)
    tweet = re.sub(r"4playthursdays", "foreplay thursdays", tweet)
    tweet = re.sub(r"tgf2015", "tonti town grape festival", tweet)
    tweet = re.sub(r"realmandyrain", "mandy rain", tweet)
    tweet = re.sub(r"graysondolan", "grayson dolan", tweet)
    tweet = re.sub(r"apollobrown", "apollo brown", tweet)
    tweet = re.sub(r"saddlebrooke", "saddle brooke", tweet)
    tweet = re.sub(r"tontitowngrape", "tonti town grape", tweet)
    tweet = re.sub(r"abbswinston", "abbs winston", tweet)
    tweet = re.sub(r"shaunKing", "sshaun king", tweet)
    tweet = re.sub(r"meekmill", "meek mill", tweet)
    tweet = re.sub(r"tornadogiveaway", "tornado giveaway", tweet)
    tweet = re.sub(r"grupdates", "gr updates", tweet)
    tweet = re.sub(r"southdowns", "south downs", tweet)
    tweet = re.sub(r"braininjury", "brain injury", tweet)
    tweet = re.sub(r"auspol", "australian politics", tweet)
    tweet = re.sub(r"PlannedParenthood", "Planned Parenthood", tweet)
    tweet = re.sub(r"calgaryweather", "calgary weather", tweet)
    tweet = re.sub(r"weallheartonedirection", "we all heart one direction", tweet)
    tweet = re.sub(r"edsheeran", "ed sheeran", tweet)
    tweet = re.sub(r"trueHeroes", "true heroes", tweet)
    tweet = re.sub(r"complexmag", "complex magazine", tweet)
    tweet = re.sub(r"theadvocatemag", "the advocate magazine", tweet)
    tweet = re.sub(r"cityofcalgary", "city of calgary", tweet)
    tweet = re.sub(r"ebolaoutbreak", "ebola outbreak", tweet)
    tweet = re.sub(r"summerfate", "summer fate", tweet)
    tweet = re.sub(r"ramag", "royal academy magazine", tweet)
    tweet = re.sub(r"offers2go", "offers to go", tweet)
    tweet = re.sub(r"modiministry", "modi ministry", tweet)
    tweet = re.sub(r"taxiways", "taxi ways", tweet)
    tweet = re.sub(r"calum5sos", "calum hood", tweet)
    tweet = re.sub(r"jamesmelville", "james melville", tweet)
    tweet = re.sub(r"jamaicaobserver", "jamaica observer", tweet)
    tweet = re.sub(r"tweetlikeitsseptember11th2001", "tweet like it is september 11th 2001", tweet)
    tweet = re.sub(r"cbplawyers", "cbp lawyers", tweet)
    tweet = re.sub(r"fewmoretweets", "few more tweets", tweet)
    tweet = re.sub(r"blacklivesmatter", "black lives matter", tweet)
    tweet = re.sub(r"nasahurricane", "nasa hurricane", tweet)
    tweet = re.sub(r"onlinecommunities", "online communities", tweet)
    tweet = re.sub(r"humanconsumption", "human consumption", tweet)
    tweet = re.sub(r"typhoon-devastated", "typhoon devastated", tweet)
    tweet = re.sub(r"meat-loving", "meat loving", tweet)
    tweet = re.sub(r"facialabuse", "facial abuse", tweet)
    tweet = re.sub(r"lakecounty", "lake county", tweet)
    tweet = re.sub(r"beingauthor", "being author", tweet)
    tweet = re.sub(r"withheavenly", "with heavenly", tweet)
    tweet = re.sub(r"thanku", "thank you", tweet)
    tweet = re.sub(r"itunesmusic", "itunes music", tweet)
    tweet = re.sub(r"offensivecontent", "offensive content", tweet)
    tweet = re.sub(r"worstsummerjob", "worst summer job", tweet)
    tweet = re.sub(r"harrybecareful", "harry be careful", tweet)
    tweet = re.sub(r"nasasolarsystem", "nasa solar system", tweet)
    tweet = re.sub(r"animalrescue", "animal rescue", tweet)
    tweet = re.sub(r"kurtschlichter", "kurt schlichter", tweet)
    tweet = re.sub(r"throwingknifes", "throwing knives", tweet)
    tweet = re.sub(r"godsLove", "god's love", tweet)
    tweet = re.sub(r"bookboost", "book boost", tweet)
    tweet = re.sub(r"ibooklove", "I book love", tweet)
    tweet = re.sub(r"nestleindia", "nestle india", tweet)
    tweet = re.sub(r"realdonaldtrump", "donald trump", tweet)
    tweet = re.sub(r"davidvonderhaar", "david vonderhaar", tweet)
    tweet = re.sub(r"ceciltheLion", "cecil the lion", tweet)
    tweet = re.sub(r"weathernetwork", "weather network", tweet)
    tweet = re.sub(r"gopdebate", "gop debate", tweet)
    tweet = re.sub(r"rickperry", "rick perry", tweet)
    tweet = re.sub(r"frontpage", "front page", tweet)
    tweet = re.sub(r"newsintweets", "news in tweets", tweet)
    tweet = re.sub(r"viralspell", "viral spell", tweet)
    tweet = re.sub(r"til_now", "until now", tweet)
    tweet = re.sub(r"volcanoinrussia", "volcano in russia", tweet)
    tweet = re.sub(r"zippednews", "Zipped news", tweet)
    tweet = re.sub(r"michelebachman", "michele bachman", tweet)
    tweet = re.sub(r"53inch", "53 inch", tweet)
    tweet = re.sub(r"kerricktrial", "kerrick trial", tweet)
    tweet = re.sub(r"abstorm", "alberta storm", tweet)
    tweet = re.sub(r"beyhive", "beyonce hive", tweet)
    tweet = re.sub(r"rockyfire", "rocky fire", tweet)
    tweet = re.sub(r"listen/buy", "listen or buy", tweet)
    tweet = re.sub(r"artistsunited", "artists united", tweet)
    tweet = re.sub(r"engvsaus", "england vs australia", tweet)
    tweet = re.sub(r"scottwalker", "scott walker", tweet)
    tweet = re.sub(r"africanbaze", "african baze", tweet)
    tweet = re.sub(r"engineshed", "engine shed", tweet)
    tweet = re.sub(r"newsnigeria", "nigeria news", tweet)
    return tweet

data_without_tockenization['text'] = data_without_tockenization['text'].apply(clean)
data_without_tockenization['location'] = data_without_tockenization['location'].apply(clean)
data_without_tockenization

,id,keyword,location,text,target
31,48,ablaze,birmingham,bbcmtd wholesale markets ablaze,1
32,49,ablaze,est september bristol,always try bring heavy metal rt,0
33,50,ablaze,africa,african baze breaking nigeria news flag set ablaze aba,1
34,52,ablaze,philadelphia pa,crying set ablaze,0
35,53,ablaze,london united kingdom,plus side look sky last night ablaze,0
...,...,...,...,...,...
7575,10826,wrecked,tn,bright side wrecked,0
7577,10829,wrecked,newcastle upon tyne united kingdom,hes gone relax thought wife wrecked cake goner mind lol whoops,0
7579,10831,wrecked,vancouver canada,three days work theyve pretty much wrecked hahaha shoutout family one,0
7580,10832,wrecked,london,fx forex trading cramer igers words wrecked disneys stock,0


In [16]:
#!pip install autocorrect
from autocorrect import Speller 

#create function to spell check strings
def spell_check(x):
    spell = Speller(lang='en')
    return " ".join([spell(i) for i in x.split()])

#showcase spellcheck 
mispelled = "pleaze don't use my toethbrush"
spell_check(mispelled)

"please don't use my toothbrush"

In [ ]:
# coding: utf-8

import nltk
from nltk.tag.stanford import StanfordNERTagger

jar = 'stanford-ner/stanford-ner.jar'
model = 'stanford-ner/english.all.3class.distsim.crf.ser.gz'

#tweets = data_without_tockenization['text']

# Prepare NER tagger with english model
ner_tagger = StanfordNERTagger(model, jar, encoding='utf8')

# Tokenize: Split sentence into words

words = []
for index, value in data_without_tockenization.iterrows():
  text = nltk.word_tokenize(value['text'])
  words.append(text)
#end for

for word in words:
  print(ner_tagger.tag(word))
#end for

[('bbcmtd', 'O'), ('wholesale', 'O'), ('markets', 'O'), ('ablaze', 'O')]
[('always', 'O'), ('try', 'O'), ('bring', 'O'), ('heavy', 'O'), ('metal', 'O'), ('rt', 'O')]
[('african', 'O'), ('baze', 'O'), ('breaking', 'O'), ('nigeria', 'O'), ('news', 'O'), ('flag', 'O'), ('set', 'O'), ('ablaze', 'O'), ('aba', 'O')]
[('crying', 'O'), ('set', 'O'), ('ablaze', 'O')]
[('plus', 'O'), ('side', 'O'), ('look', 'O'), ('sky', 'O'), ('last', 'O'), ('night', 'O'), ('ablaze', 'O')]
[('phdsquares', 'O'), ('mufc', 'O'), ('theyve', 'O'), ('built', 'O'), ('much', 'O'), ('hype', 'O'), ('around', 'O'), ('new', 'O'), ('acquisitions', 'O'), ('doubt', 'O'), ('set', 'O'), ('epl', 'O'), ('ablaze', 'O'), ('season', 'O')]
[('inec', 'O'), ('office', 'O'), ('abia', 'O'), ('set', 'O'), ('ablaze', 'O')]
[('ablaze', 'O'), ('lord', 'O')]
[('check', 'O'), ('nsfw', 'O')]
[('awesome', 'O'), ('time', 'O'), ('visiting', 'O'), ('cfc', 'O'), ('head', 'O'), ('office', 'O'), ('ancop', 'O'), ('site', 'O'), ('ablaze', 'O'), ('thanks

[('norwaymfa', 'O'), ('bahrain', 'O'), ('police', 'O'), ('previously', 'O'), ('died', 'O'), ('road', 'O'), ('accident', 'O'), ('killed', 'O'), ('explosion', 'O')]
[('still', 'O'), ('heard', 'O'), ('church', 'O'), ('leaders', 'O'), ('kenew', 'O'), ('yorka', 'O'), ('coming', 'O'), ('forward', 'O'), ('comment', 'O'), ('accident', 'O'), ('issue', 'O'), ('disciplinary', 'O'), ('measuresarrestpastornganga', 'O')]
[('aftershockdelo', 'O'), ('scuf', 'O'), ('ps', 'O'), ('live', 'O'), ('game', 'O'), ('cya', 'O')]
[('man', 'O'), ('drive', 'O'), ('effort', 'O'), ('gets', 'O'), ('painful', 'O'), ('man', 'O'), ('win', 'O'), ('roger', 'O'), ('bannister', 'O')]
[('icemoon', 'O'), ('dj', 'O'), ('icemoon', 'O'), ('dubstep', 'O'), ('trap', 'O'), ('music', 'O'), ('dnb', 'O'), ('edm', 'O'), ('dance', 'O'), ('ices', 'O'), ('û', 'O')]
[('victory', 'O'), ('bargain', 'O'), ('basement', 'O'), ('prices', 'O'), ('dwight', 'O'), ('david', 'PERSON'), ('eisenhower', 'PERSON')]
[('icemoon', 'O'), ('dj', 'O'), ('icemo

[('airplane', 'O'), ('accident', 'O')]
[('phone', 'O'), ('looks', 'O'), ('like', 'O'), ('car', 'O'), ('ship', 'O'), ('airplane', 'O'), ('accident', 'O'), ('terrible', 'O')]
[('statistically', 'O'), ('im', 'O'), ('risk', 'O'), ('getting', 'O'), ('killed', 'O'), ('cop', 'O'), ('dying', 'O'), ('airplane', 'O'), ('accident', 'O')]
[('airplane', 'O'), ('crashes', 'O'), ('house', 'O'), ('colombia', 'O'), ('people', 'O'), ('die', 'O'), ('accident', 'O')]
[('shooting', 'O'), ('airplane', 'O'), ('accident', 'O')]
[('could', 'O'), ('drone', 'O'), ('cause', 'O'), ('airplane', 'O'), ('accident', 'O'), ('pilots', 'O'), ('worried', 'O'), ('use', 'O'), ('drones', 'O'), ('esp', 'O'), ('close', 'O'), ('vicinity', 'O'), ('airports', 'O')]
[('twelve', 'O'), ('feared', 'O'), ('killed', 'O'), ('pakistani', 'O'), ('air', 'O'), ('ambulance', 'O'), ('helicopter', 'O'), ('crash', 'O')]
[('two', 'O'), ('air', 'O'), ('ambulances', 'O'), ('scene', 'O'), ('serious', 'O'), ('crash', 'O'), ('two', 'O'), ('cars', 'O'

[('starmade', 'O'), ('stardate', 'O'), ('planetary', 'O'), ('annihilation', 'O'), ('via', 'O'), ('youtube', 'O')]
[('us', 'O'), ('national', 'O'), ('park', 'O'), ('services', 'O'), ('tonto', 'O'), ('national', 'O'), ('forest', 'O'), ('stop', 'O'), ('annihilation', 'O'), ('salt', 'O'), ('river', 'O'), ('wild', 'O'), ('horse', 'O'), ('via', 'O'), ('change', 'O')]
[('kimkardashian', 'O'), ('please', 'O'), ('sign', 'O'), ('share', 'O'), ('petition', 'O'), ('save', 'O'), ('wild', 'O'), ('horses', 'O'), ('arizona', 'O')]
[('theellenshow', 'O'), ('please', 'O'), ('check', 'O'), ('salt', 'O'), ('river', 'O'), ('horses', 'O'), ('help', 'O'), ('stop', 'O'), ('annihilation', 'O'), ('happen', 'O'), ('without', 'O'), ('signatureschange', 'O'), ('org', 'O'), ('thx', 'O')]
[('souls', 'O'), ('punished', 'O'), ('withåêannihilation', 'O')]
[('reject', 'O'), ('laws', 'O'), ('misguided', 'O'), ('false', 'O'), ('prophets', 'O'), ('imprison', 'O'), ('nations', 'O'), ('fueling', 'O'), ('self', 'O'), ('annihi

[('check', 'O'), ('preppers', 'O'), ('doomsday', 'O'), ('must', 'O'), ('library', 'O'), ('collection', 'O'), ('cd', 'O'), ('shtf', 'O'), ('preppertalk', 'O'), ('survival', 'O'), ('prepper', 'O'), ('via', 'O'), ('ebay', 'O')]
[('erker', 'O'), ('eep', 'O'), ('thought', 'O'), ('yesterday', 'O'), ('saw', 'O'), ('hella', 'O'), ('scary', 'O'), ('hail', 'O'), ('armageddon', 'O')]
[('sadly', 'O'), ('windows', 'O'), ('reveals', 'O'), ('microsofts', 'O'), ('ethics', 'O'), ('armageddon', 'O')]
[('armageddon', 'O'), ('averted', 'O'), ('el', 'O'), ('patronultimalucha', 'O')]
[('samihonkonen', 'O'), ('time', 'O'), ('hours', 'O'), ('latest', 'O'), ('series', 'O'), ('blueprint', 'O'), ('armageddon', 'O'), ('extremely', 'O'), ('impressive', 'O')]
[('european', 'O'), ('fitba', 'O'), ('till', 'O'), ('christmas', 'O'), ('armageddon', 'O')]
[('official', 'O'), ('vid', 'O'), ('gt', 'O'), ('doublecups', 'O'), ('gtgt', 'O'), ('gtgt', 'O'), ('trubgme', 'O'), ('prod', 'O'), ('thisizbwright', 'O'), ('gtgt', 'O')

...

[('tubestrike', 'O'), ('tfl', 'O'), ('workers', 'O'), ('may', 'O'), ('trouble', 'O'), ('planning', 'O'), ('downtime', 'O'), ('hope', 'O'), ('none', 'O'), ('need', 'O'), ('emergency', 'O'), ('services', 'O')]
[('breaking', 'O'), ('news', 'O'), ('call', 'O'), ('tasmanias', 'O'), ('emergency', 'O'), ('services', 'O'), ('trained', 'O'), ('horse', 'O')]
[('recommend', 'O'), ('anew', 'O'), ('yorkone', 'O'), ('job', 'O'), ('rn', 'O'), ('emergency', 'O'), ('services', 'O'), ('full', 'O'), ('time', 'O'), ('rose', 'O'), ('de', 'O'), ('lima', 'O'), ('campus', 'O'), ('hiring', 'O')]
[('healthcare', 'O'), ('job', 'O'), ('kodiak', 'O'), ('ak', 'O'), ('emergency', 'O'), ('services', 'O'), ('supervisor', 'O'), ('emergency', 'O'), ('providence', 'O'), ('health', 'O'), ('amp', 'O'), ('services', 'O'), ('jobs', 'O')]
[('air', 'O'), ('group', 'O'), ('rescue', 'O'), ('emergency', 'O'), ('service', 'O'), ('learn', 'O')]
[('want', 'O'), ('work', 'O'), ('swedish', 'O'), ('health', 'O'), ('services', 'O'), ('h

[('jay', 'O'), ('alexis', 'O'), ('broke', 'O'), ('goes', 'O'), ('faith', 'O'), ('goals', 'O'), ('people', 'O'), ('evacuate', 'O')]
[('joonma', 'O'), ('dealbreaker', 'O'), ('thats', 'O'), ('thats', 'O'), ('dealbreaker', 'O'), ('sos', 'O'), ('abandon', 'O'), ('ship', 'O'), ('evacuate', 'O'), ('building', 'O')]
[('stores', 'O'), ('fire', 'O'), ('alarms', 'O'), ('went', 'O'), ('today', 'O'), ('work', 'O'), ('evacuate', 'O'), ('like', 'O'), ('omg', 'O'), ('isis', 'O'), ('happening', 'O')]
[('dont', 'O'), ('evacuate', 'O'), ('students', 'O'), ('throw', 'O'), ('dungeon', 'O'), ('stupid', 'O')]
[('mygc', 'O'), ('broken', 'O'), ('powerlines', 'O'), ('evacuate', 'O'), ('gold', 'O'), ('coast', 'O'), ('tram', 'O'), ('suspend', 'O'), ('services', 'O')]
[('us', 'O'), ('govt', 'O'), ('refuses', 'O'), ('evacuate', 'O'), ('americans', 'O'), ('yemen', 'O'), ('yemen', 'O'), ('united', 'O'), ('states', 'O'), ('of', 'O'), ('america', 'O'), ('evacuation', 'O'), ('abandoned', 'O')]
[('ahhtheenikki', 'O'), ('

[('elem', 'O'), ('pomo', 'O'), ('helping', 'O'), ('displaced', 'O'), ('rocky', 'O'), ('fire', 'O'), ('please', 'O'), ('considerelem', 'O'), ('evacuation', 'O'), ('center', 'O'), ('via', 'O'), ('gofundme', 'O')]
[('bend', 'O'), ('post', 'O'), ('office', 'O'), ('roofers', 'O'), ('cut', 'O'), ('gas', 'O'), ('line', 'O'), ('prompt', 'O'), ('evacuation', 'O')]
[('faan', 'O'), ('orders', 'O'), ('evacuation', 'O'), ('abandoned', 'O'), ('aircraft', 'O'), ('mma', 'O'), ('faan', 'O'), ('noted', 'O'), ('action', 'O'), ('become', 'O'), ('neces', 'O'), ('via', 'O'), ('todayngr', 'O')]
[('ericbulak', 'O'), ('jaclynsonne', 'O'), ('oliviaann', 'O'), ('looking', 'O'), ('guys', 'O'), ('live', 'O'), ('stream', 'O'), ('im', 'O'), ('guessing', 'O'), ('evacuation', 'O'), ('cost', 'O'), ('front', 'O')]
[('faan', 'O'), ('gives', 'O'), ('owners', 'O'), ('abandoned', 'O'), ('aircraft', 'O'), ('evacuation', 'O'), ('ultimatum', 'O'), ('via', 'O'), ('dailytimesngr', 'O'), ('probe', 'O')]
[('updated', 'O'), ('rocky

[('saw', 'O'), ('man', 'O'), ('whose', 'O'), ('mind', 'O'), ('exploded', 'O'), ('drako', 'O'), ('zarharzar', 'O'), ('day', 'O')]
[('told', 'O'), ('ten', 'O'), ('years', 'O'), ('ago', 'O'), ('id', 'O'), ('seeing', 'O'), ('anime', 'O'), ('big', 'O'), ('screen', 'O'), ('probably', 'O'), ('would', 'O'), ('exploded', 'O')]
[('lunited', 'O'), ('kingdome', 'O'), ('microphone', 'O'), ('exploded', 'O'), ('ovaries', 'O')]
[('holy', 'O'), ('crap', 'O'), ('phone', 'O'), ('exploded', 'O'), ('haha', 'O')]
[('man', 'O'), ('squeeze', 'O'), ('another', 'O'), ('mans', 'O'), ('head', 'O'), ('bare', 'O'), ('hands', 'O'), ('literally', 'O'), ('exploded', 'O')]
[('itsnasb', 'O'), ('go', 'O'), ('replace', 'O'), ('sarcasm', 'O'), ('meter', 'O'), ('exploded', 'O')]
[('kai', 'O'), ('forbath', 'O'), ('demolished', 'O'), ('weather', 'O'), ('station', 'O'), ('set', 'O'), ('drill', 'O'), ('field', 'O'), ('missed', 'O'), ('field', 'O'), ('goal', 'O'), ('thing', 'O'), ('exploded', 'O'), ('metal', 'O'), ('bits', 'O')]

[('russia', 'O'), ('food', 'O'), ('crematoria', 'O'), ('provoke', 'O'), ('outrage', 'O'), ('country', 'O'), ('wsoaring', 'O'), ('poverty', 'O'), ('soviet', 'O'), ('famine', 'O'), ('memory', 'O')]
[('ppl', 'O'), ('like', 'O'), ('unu', 'O'), ('feast', 'O'), ('today', 'O'), ('famine', 'O'), ('tomorrow', 'O')]
[('top', 'O'), ('turn', 'O'), ('darkness', 'O'), ('king', 'O'), ('redeem', 'O'), ('queen', 'O'), ('serene', 'O'), ('famine', 'O'), ('wolf', 'O')]
[('memorial', 'O'), ('millions', 'O'), ('perished', 'O'), ('holodomor', 'O'), ('erected', 'O'), ('us', 'O'), ('capital', 'O')]
[('exporting', 'O'), ('food', 'O'), ('wont', 'O'), ('solve', 'O'), ('problem', 'O'), ('africans', 'O'), ('end', 'O'), ('famine', 'O'), ('n', 'O'), ('poverty', 'O'), ('solving', 'O'), ('control', 'O'), ('tribal', 'O'), ('wars', 'O')]
[('building', 'O'), ('muscle', 'O'), ('feast', 'O'), ('famine', 'O'), ('feeding', 'O'), ('diet', 'O'), ('plans', 'O'), ('based', 'O'), ('around', 'O'), ('losing', 'O'), ('weight', 'O'), 